# Single A2A Example

This notebook demonstrates how to run a single Allocation-to-Allocators (A2A) job from the AlloOptim library.

# Optimizer Name Standardization Plan

## Name Mapping: Old → New

### CMA Family
- `CMA_MEAN_VARIANCE` → `CMAMeanVariance`
- `CMA_L_MOMENTS` → `CMALMoments`
- `CMA_SORTINO` → `CMASortino`
- `CMA_MAX_DRAWDOWN` → `CMAMaxDrawdown`
- `CMA_ROBUST_SHARPE` → `CMARobustSharpe`
- `CMA_CVAR` → `CMACVar`

### PSO Family
- `PSO_MeanVariance` → `PSOMeanVariance`
- `PSO_LMoments` → `PSOLMoments`

### Adjusted Returns Family
- `AdjustedReturns_MeanVariance` → `AdjustedReturnsMeanVariance`
- `AdjustedReturns_LMoments` → `AdjustedReturnsLMoments`
- `AdjustedReturns_EMA` → `AdjustedReturnsEMA`
- `AdjustedReturns_SemiVariance` → `AdjustedReturnsSemiVariance`

### Simple/Naive Optimizers
- `Naive` → `NaiveOptimizer`
- `CappedMomentum` → `CappedMomentumOptimizer`
- `EMACappedMomentum` → `EMACappedMomentumOptimizer`

### Hierarchical/Clustering
- `HRP` → `HRPOptimizer`
- `NCOSharpeOptimizer` → ✅ Keep as is
- `NCOVarianceOptimizer` → ✅ Keep as is

### Risk-Based
- `RiskParity` → `RiskParityOptimizer`
- `RobustMVO` → `RobustMeanVarianceOptimizer`

### Efficient Frontier
- `MaxSharpe` → ✅ Keep as is
- `EfficientRisk` → ✅ Keep as is
- `EfficientReturn` → ✅ Keep as is
- `BlackLittermannOptimizer` → `BlackLittermanOptimizer` (typo fix)

### Advanced Methods
- `HigherMoment` → `HigherMomentOptimizer`
- `KellyCriterion` → `KellyCriterionOptimizer`

### Machine Learning
- `LightGBMOptimizer` → ✅ Keep as is
- `AugmentedLightGBMOptimizer` → ✅ Keep as is
- `LSTMOptimizer` → ✅ Keep as is
- `MAMBAOptimizer` → ✅ Keep as is
- `TCNOptimizer` → ✅ Keep as is

### Other
- `WikipediaOptimizer` → ✅ Keep as is
- `BalancedFundamentalOptimizer` → ✅ Keep as is
- `QualityGrowthFundamentalOptimizer` → ✅ Keep as is
- `ValueInvestingFundamentalOptimizer` → ✅ Keep as is
- `MarketCapFundamentalOptimizer` → ✅ Keep as is

### Ensemble/Benchmark
- `A2A_Ensemble` → `A2AEnsemble`
- `SPY_Benchmark` → `SPYBenchmark`

---

## Implementation Plan

### Phase 1: Preparation & Testing Infrastructure
**Goal:** Ensure all tests pass with current names and create compatibility layer

**Tasks:**
1. Run full test suite to establish baseline (`pytest`)
2. Document all current usages in config files, notebooks, and examples
3. Create a name mapping dictionary in a central configuration file
4. Add backward compatibility utility function for name translation

**Duration:** 1 day

---

### Phase 2: Core Optimizer Name Updates
**Goal:** Update the `@property def name()` methods in optimizer classes

**Tasks:**
1. Update CMA family optimizers (6 files)
   - cma_optimizer.py - update `return f"CMA_{...}"` pattern
2. Update PSO family optimizers (1 file)
   - pso_optimizer.py
3. Update Adjusted Returns family (1 file)
   - adjusted_return_optimizer.py
4. Update simple optimizers (3 files)
   - naive_optimizer.py
   - momentum_optimizer.py
   - hrp_optimizer.py
5. Update risk-based optimizers (2 files)
   - risk_parity_optimizer.py
   - robust_mean_variance_optimizer.py
6. Update advanced methods (2 files)
   - higher_moments_optimizer.py
   - kelly_criterion_optimizer.py
7. Fix typo in Black-Litterman (1 file)
   - black_litterman_optimizer.py

**Duration:** 2-3 hours

---

### Phase 3: Update Test Files & Configuration
**Goal:** Update all references in tests, configs, and example notebooks

**Tasks:**
1. Update test files
   - `test_allocation_optimizers.py`
   - `test_optimizer.py`
   - `test_ml_optimizers.py`
   - `test_a2a.py`
   - Any other test files with optimizer name references
2. Update configuration files
   - `BacktestConfig` default optimizer lists
   - Any YAML/JSON config files
3. Update example files
   - comprehensive_backtest.py
   - `single_a2a_example.ipynb`
   - Any other example notebooks

**Duration:** 2-3 hours

---

### Phase 4: Update Ensemble & Factory Code
**Goal:** Update ensemble and factory pattern code that references optimizer names

**Tasks:**
1. Update optimizer_list.py - optimizer registry
2. Update ensemble_optimizers.py - A2A and benchmark names
3. Update orchestrator files if they have hardcoded names
4. Update any string matching or filtering logic

**Duration:** 1-2 hours

---

### Phase 5: Documentation & Results Files
**Goal:** Update documentation and handle old result files

**Tasks:**
1. Update README.md with new optimizer names
2. Update docstrings and comments
3. Add migration notes for users with old config files
4. Update result parsing to handle both old and new names (for reading historical backtests)
5. Create a migration script for users with existing configurations

**Duration:** 2-3 hours

---

### Phase 6: Testing & Validation
**Goal:** Comprehensive testing to ensure nothing broke

**Tasks:**
1. Run full pytest suite
2. Run comprehensive backtest with new names
3. Test all example notebooks
4. Verify backward compatibility layer works
5. Test result file loading with old and new names
6. Performance regression testing

**Duration:** 2-3 hours

---

### Phase 7: Cleanup & Deprecation
**Goal:** Remove backward compatibility after grace period

**Tasks:**
1. Keep backward compatibility for 1-2 releases
2. Add deprecation warnings when old names are used
3. Eventually remove translation layer (optional, in future release)
4. Update CHANGELOG.md with breaking changes

**Duration:** 1 hour (initial), ongoing for future releases

---

## Total Estimated Time

**Total Duration:** 1-2 days of focused work

**Breakdown:**
- Core changes: 4-6 hours
- Testing & validation: 4-6 hours  
- Documentation: 2-3 hours
- Buffer for unexpected issues: 2-4 hours

**Risk Level:** Medium
- Many files affected, but changes are straightforward
- Main risk: Missing references in config/test files
- Mitigation: Comprehensive grep search before and after changes

---

## Naming Convention Summary

**Pattern:** `{Strategy}{Variant}Optimizer`

**Rules:**
1. No underscores - use pure CamelCase
2. Add `Optimizer` suffix to short acronyms (HRP, MVO, etc.)
3. Keep descriptive names for complex strategies
4. Acronyms in uppercase (CMA, PSO, EMA, HRP, NCO)
5. Full words in CamelCase (MeanVariance, LMoments, etc.)

## 1. Import Libraries

Import the necessary libraries for A2A optimization.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime

# AlloOptim imports
from allo_optim.allocation_to_allocators.allocation_orchestrator import (
    AllocationOrchestrator,
    AllocationOrchestratorConfig,
    OrchestrationType,
)
from allo_optim.config.stock_universe import list_of_dax_stocks

print("Libraries imported successfully!")

## 2. Create Sample Portfolio Data

Create sample price data for a portfolio of assets.

In [ ]:
# Set random seed for reproducibility
np.random.seed(42)

# Get sample stocks (using first 5 DAX stocks)
all_stocks = list_of_dax_stocks()[:5]
assets = [stock.symbol for stock in all_stocks]

# Generate 6 months of daily price data
dates = pd.date_range("2023-01-01", periods=126, freq="D")
n_assets = len(assets)

# Generate correlated price movements
returns = np.random.multivariate_normal(
    mean=np.full(n_assets, 0.0005),  # 0.05% daily return
    cov=np.eye(n_assets) * 0.0004 + np.full((n_assets, n_assets), 0.0001),  # Some correlation
    size=len(dates),
)

# Create price DataFrame
prices = pd.DataFrame(
    100 * np.exp(np.cumsum(returns, axis=0)),
    index=dates,
    columns=assets
)

print("Sample Data Created:")
print(f"Assets: {', '.join(assets)}")
print(f"Date range: {dates[0].date()} to {dates[-1].date()}")
print(f"Price data shape: {prices.shape}")
print(f"\nSample prices (first 3 rows):")
print(prices.head(3))

## 3. Run A2A Optimization

Initialize and run the Allocation-to-Allocators optimization with equal weighting.

In [ ]:
# Configure A2A with 3 optimizers
optimizer_names = ["MaxSharpe", "RiskParityOptimizer", "NaiveOptimizer"]

## 4. Display Results

Show the final portfolio allocation and optimizer weights.

In [ ]:
if result.success:
    # Display asset weights
    print("Final Portfolio Allocation:")
    print("-" * 40)
    sorted_weights = sorted(result.asset_weights.items(), key=lambda x: x[1], reverse=True)
    for asset, weight in sorted_weights:
        print(f"{asset}: {weight:.4f} ({weight*100:.2f}%)")

    total_weight = sum(result.asset_weights.values())
    print(f"\nTotal weight: {total_weight:.6f}")

    # Display optimizer weights
    if hasattr(result.statistics, 'algo_weights'):
        print(f"\nOptimizer Weights:")
        print("-" * 40)
        for optimizer, weight in result.statistics.algo_weights.items():
            print(f"{optimizer}: {weight:.4f} ({weight*100:.2f}%)")

    # Create visualization
    plt.figure(figsize=(12, 5))

    # Asset allocation bar chart
    plt.subplot(1, 2, 1)
    assets_list = [asset for asset, _ in sorted_weights]
    weights_list = [weight for _, weight in sorted_weights]
    plt.bar(assets_list, weights_list, color='skyblue')
    plt.title('Asset Allocation')
    plt.xlabel('Assets')
    plt.ylabel('Weight')
    plt.xticks(rotation=45)
    plt.grid(axis='y', alpha=0.3)

    # Optimizer weights bar chart
    plt.subplot(1, 2, 2)
    if hasattr(result.statistics, 'algo_weights'):
        optimizers = list(result.statistics.algo_weights.keys())
        opt_weights = list(result.statistics.algo_weights.values())
        plt.bar(optimizers, opt_weights, color='lightgreen')
        plt.title('Optimizer Weights')
        plt.xlabel('Optimizers')
        plt.ylabel('Weight')
        plt.xticks(rotation=45)
        plt.grid(axis='y', alpha=0.3)

    plt.tight_layout()
    plt.show()

    print("✅ A2A optimization completed successfully!")

else:
    print("❌ A2A optimization failed")
    if hasattr(result, 'error_message'):
        print(f"Error: {result.error_message}")